# L1 Language Models, the Chat Format and Tokens

## Setup
#### Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [1]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"))

deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

In [2]:
def get_completion(prompt,deployment_name=deployment_name,temperature=0.01):
    completion = client.chat.completions.create(
        model=deployment_name,
        max_tokens=800,
        temperature=temperature,# this is the degree of randomness of the model's output
        messages= [{"role": "user", "content": prompt}]
        )
    return completion.choices[0].message.content

## Prompt the model and get a completion

In [3]:
response = get_completion("What is the capital of France?")

In [4]:
print(response)

The capital of France is Paris.


## Tokens

In [5]:
response = get_completion("Take the letters in lollipop \
and reverse them")
print(response)

ppilolol


"lollipop" in reverse should be "popillol"

In [6]:
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")

In [7]:
print(response)

p-o-p-i-l-l-o-l


## Helper function (chat format)
Here's the helper function we'll use in this course.

In [8]:
def get_completion_from_messages(messages,deployment_name=deployment_name,temperature=0.01):
    completion = client.chat.completions.create(
        model=deployment_name,
        max_tokens=800,
        temperature=temperature,# this is the degree of randomness of the model's output
        messages= messages
        )
    return completion.choices[0].message.content

In [9]:
messages =  [  
{'role':'system', 
 'content':"""You are an assistant who\
 responds in the style of Dr Seuss."""},    
{'role':'user', 
 'content':"""write me a very short poem\
 about a happy carrot"""},  
] 

print(messages)

[{'role': 'system', 'content': 'You are an assistant who responds in the style of Dr Seuss.'}, {'role': 'user', 'content': 'write me a very short poem about a happy carrot'}]


In [10]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

Oh, happy carrot, how you shine,
With a bright orange color that's simply divine.
Your crispness and crunch are a tasty delight,
And with every bite, my smile shines bright.


In [11]:
# length
messages =  [  
{'role':'system',
 'content':'All your responses must be \
one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
] 

print(messages)

[{'role': 'system', 'content': 'All your responses must be one sentence long.'}, {'role': 'user', 'content': 'write me a story about a happy carrot'}]


In [12]:
response = get_completion_from_messages(messages, temperature =1)
print(response)

Once there was a carrot named Carl who lived in a lush vegetable garden and was always content, basking in the warm sun, drinking in the rain, and chatting with his fellow plants.


In [13]:
# combined
messages =  [  
{'role':'system',
 'content':"""You are an assistant who \
responds in the style of Dr Seuss. \
All your responses must be one sentence long."""},    
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
] 

print(messages)

[{'role': 'system', 'content': 'You are an assistant who responds in the style of Dr Seuss. All your responses must be one sentence long.'}, {'role': 'user', 'content': 'write me a story about a happy carrot'}]


In [14]:
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

Once there was a happy carrot named Carl, who loved nothing more than growing tall.


In [34]:
def get_completion_and_token_count(messages,deployment_name=deployment_name,temperature=0.01):
    
    response = client.chat.completions.create(
        model=deployment_name,
        max_tokens=800,
        temperature=temperature,# this is the degree of randomness of the model's output
        messages= messages
        )

    content = response.choices[0].message.content
    token_dict = {
'prompt_tokens':response.usage.prompt_tokens,
'completion_tokens':response.usage.completion_tokens,
'total_tokens':response.usage.total_tokens,
    }
    return content, token_dict

In [16]:
messages = [
{'role':'system', 
 'content':"""You are an assistant who responds\
 in the style of Dr Seuss."""},    
{'role':'user',
 'content':"""write me a very short poem \ 
 about a happy carrot"""},  
] 
print(messages)

[{'role': 'system', 'content': 'You are an assistant who responds in the style of Dr Seuss.'}, {'role': 'user', 'content': 'write me a very short poem \\ \n about a happy carrot'}]


In [35]:
response, token_dict = get_completion_and_token_count(messages)
print(response)

Oh, happy carrot, orange and bright,
Growing tall with all your might,
In the garden, you bring delight,
A healthy snack, oh what a sight!


In [36]:
print(token_dict)

{'prompt_tokens': 39, 'completion_tokens': 33, 'total_tokens': 72}


In [32]:
token_dict.usage.prompt_tokens

39

#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.